In [1]:
import os
import sys
import shutil
from pathlib import Path
from collections import defaultdict
from multiprocessing import Pool

In [2]:
sys.path.append('../src')
from utils import run_cmd

In [3]:
raw_files = """/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-5595_S16_L001_R1_001.fastq.gz
/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-5595_S16_L001_R2_001.fastq.gz
/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-5655_S17_L001_R1_001.fastq.gz
/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-5655_S17_L001_R2_001.fastq.gz
/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-5802_S18_L001_R1_001.fastq.gz
/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-5802_S18_L001_R2_001.fastq.gz
/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-6098_S19_L001_R1_001.fastq.gz
/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-6098_S19_L001_R2_001.fastq.gz
/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-6160_S20_L001_R1_001.fastq.gz
/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-6160_S20_L001_R2_001.fastq.gz
/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-6166_S21_L001_R1_001.fastq.gz
/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-6166_S21_L001_R2_001.fastq.gz
"""

In [4]:
raw_files = raw_files.split()
pairwise = defaultdict(list)
for raw in raw_files:
    pairwise[Path(raw).stem.split('_')[0]].append(raw)
pairwise

defaultdict(list,
            {'R17-5595': ['/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-5595_S16_L001_R1_001.fastq.gz',
              '/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-5595_S16_L001_R2_001.fastq.gz'],
             'R17-5655': ['/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-5655_S17_L001_R1_001.fastq.gz',
              '/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-5655_S17_L001_R2_001.fastq.gz'],
             'R17-5802': ['/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-5802_S18_L001_R1_001.fastq.gz',
              '/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-5802_S18_L001_R2_001.fastq.gz'],
             'R17-6098': ['/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH/Data/Intensities/BaseCalls/R17-6098_S19_L001_R1_001.fastq.gz',
       

In [5]:
outpath = Path('/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH')
program = '/media/Central_Lab_Storage/bo-han/Projects/MiAW/miaw.py'

cmds = []
for sample_name, (sr_1, sr_2) in pairwise.items():
    cmd = f"conda run -n miaw python {program} -1 {sr_1} -2 {sr_2} -o {outpath/sample_name} -t 8"
    cmds.append(cmd)

In [9]:
with Pool(8) as p:
    try:
        p.map(run_cmd, cmds)
        p.close()
        p.join()
    except KeyboardInterrupt:
        p.terminate()

In [11]:
dirpath = Path('/media/MiSeqOutput/180312_M04855_0023_000000000-BM7VH')
outpath = Path('/media/GenomicResearch/MiSeq/Listeria_monocytogenes/New')

for i in dirpath.iterdir():
    src = i/'assembly.fasta'
    dst = outpath/(i.name + '.fa')
    if src.exists():
        shutil.copyfile(src, dst)